# Finding Anomalies in Bitcoin Price Data

### VAE EXPLANATION:



In [94]:
#Imports
import pandas as pd
import datetime as dt
import numpy as np

In [95]:
#Importing btc stock data
data = pd.read_csv('bit_data.csv', parse_dates=['Date']) 
data.head()

        Date    High     Low    Last     Bid     Ask        Volume    VWAP
0 2014-04-15  515.00  453.16  499.01  500.01  505.04  28535.844106  491.41
1 2014-04-16  548.00  494.02  534.00  535.01  536.00  31159.941300  520.21
2 2014-04-17  537.24  481.63  506.52  504.70  505.38  21126.375080  504.83
3 2014-04-18  508.43  470.00  487.00  484.14  487.00  11879.484756  485.72
4 2014-04-19  507.43  472.81  504.74  504.74  505.00  10262.195861  492.22


In [96]:
#Generating Additional features
data['Day']=data['Date'].dt.day
data['Day_of_week']=data['Date'].dt.weekday
data['Raw_Volume']=data['Volume']
data['Raw_Last']=data['Last']
data['Raw_VWAP']=data['VWAP']
data.head()

In [98]:
#Adding moving averages
close_avg_3 = data['Last'].rolling(3).mean()
close_avg_5 =  data['Last'].rolling(5).mean()
close_avg_10 = data['Last'].rolling(10).mean()

data['3rd Closing Avg']=close_avg_3
data['5th Closing Avg']=close_avg_5
data['10th Closing Avg']=close_avg_10

In [99]:
#Removing NAN valued rows
data = data.dropna(how='any')

In [100]:
# Features to be trained on
features = [f for f in list(data) if f not in ['Date','Raw_Volume','Raw_Last','Raw_VWAP']]

In [101]:
df = data.copy()

# Normalize column data between 0 and 1
df[features] = df[features].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

print(features)

['High', 'Low', 'Last', 'Bid', 'Ask', 'Volume', 'VWAP', 'Day', 'Day_of_week', '3rd Closing Avg', '5th Closing Avg', '10th Closing Avg']


In [102]:
df.head()

         Date      High       Low      Last       Bid       Ask    Volume  \
9  2014-04-24  0.014710  0.017889  0.017248  0.017172  0.017156  0.050171   
10 2014-04-25  0.015019  0.015599  0.015421  0.015399  0.015327  0.257063   
11 2014-04-26  0.013269  0.016310  0.015183  0.015215  0.015143  0.082784   
12 2014-04-27  0.012807  0.015644  0.014547  0.014578  0.014457  0.054306   
13 2014-04-28  0.012302  0.014831  0.014554  0.014585  0.014486  0.147186   

        VWAP       Day  Day_of_week    Raw_Volume  Raw_Last  Raw_VWAP  \
9   0.015677  0.766667     0.500000   6875.473428    498.32    488.46   
10  0.014382  0.800000     0.666667  32311.719807    463.54    463.95   
11  0.014200  0.833333     0.833333  10885.104866    459.00    460.51   
12  0.013577  0.866667     1.000000   7383.826598    446.90    448.73   
13  0.013001  0.900000     0.000000  18802.958300    447.03    437.82   

    3rd Closing Avg  5th Closing Avg  10th Closing Avg  
9          0.015814         0.016185     

In [103]:
# Train-Test split
from sklearn.model_selection import train_test_split
features = [f for f in list(df)]
X_train, X_test = train_test_split(df[features], test_size=0.20, random_state=26)

In [104]:
features = [f for f in list(data) if f not in ['Date','Raw_Volume','Raw_Last','Raw_VWAP']]
X_test = X_test[features]
X_train = X_train[features]

print("X train shape: ", X_train.shape)
print("X test shape: ", X_test.shape)

X shape:  (1294, 12) (324, 12)


In [105]:
# Batch processing
m = 50

num_batches_train = X_train.shape[0]//m
print(num_batches_train)
X_train_trun = X_train.head(num_batches_train*m)
print(X_train_trun.shape)

num_batches_test = X_test.shape[0]//m
X_test_trun = X_test.head(num_batches_test*m)
print(X_test_trun.shape)

25
(1250, 12)
(300, 12)


In [106]:
import torch
from torch import nn
from torch.autograd import Variable

# Variables
batch_size = 50
representation_size = 2
encoder_dim = 6
decoder_dim = 6
decoder_out_dim = X_train_trun.shape[1] 
n_x = X_train_trun.shape[1]

In [107]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.en1 = nn.Linear(n_x, encoder_dim)
        self.en_mu = nn.Linear(encoder_dim, representation_size)
        self.en_std = nn.Linear(encoder_dim, representation_size)
        self.de1 = nn.Linear(representation_size, decoder_dim)
        self.de2 = nn.Linear(decoder_dim, decoder_out_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def encode(self, x):
        #Encode a batch of samples, and return posterior parameters for each point
        h1 = self.relu(self.en1(x))
        return self.en_mu(h1), self.en_std(h1)
    
    def decode(self, z):
        #Decode a batch of latent variables
        
        h2 = self.relu(self.de1(z))
        return self.sigmoid(self.de2(h2))
    
    def reparam(self, mu, logvar):
        #Reparameterisation trick to sample z values. 
        #This is stochastic during training, and returns the mode during evaluation.
        
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
            
    
    def forward(self, x):
        #Takes a batch of samples, encodes them, and then decodes them again to compare.
        mu, logvar = self.encode(x.contiguous().view(-1, n_x))
        z = self.reparam(mu, logvar)
        return self.decode(z), mu, logvar
    
    def loss(self, reconstruction, x, mu, logvar):
        #ELBO assuming entries of x are binary variables, with closed form KLD
        
        bce = torch.nn.functional.binary_cross_entropy(reconstruction, x.contiguous().view(-1, n_x))
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        # Normalise by same number of elements as in reconstruction
        KLD /= x.contiguous().view(-1, n_x).data.shape[0] * n_x

        return bce + KLD
    
    def rget_z(self, x):
        #Encode a batch of data points, x, into their z representations.
        
        mu, logvar = self.encode(x.contiguous().view(-1,n_x))
        return self.reparam(mu, logvar)

In [108]:
model =VAE()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model

VAE(
  (en1): Linear(in_features=12, out_features=6, bias=True)
  (en_mu): Linear(in_features=6, out_features=2, bias=True)
  (en_std): Linear(in_features=6, out_features=2, bias=True)
  (de1): Linear(in_features=2, out_features=6, bias=True)
  (de2): Linear(in_features=6, out_features=12, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [109]:
#Training model
def train(epoch, batches_per_epoch = 501, log_interval=500):
    model.train()
    ind = np.arange(X_train_trun.shape[0]) # Get array of nos.
    
    for i in range(batches_per_epoch):
        d = X_train_trun.iloc[[x for x in ind],:]
        data = torch.Tensor(d.values)
        data = Variable(data, requires_grad=False)
        
        optimizer.zero_grad() # Make gradients zero before starting to accumulate them
        
        recon_batch, mu, logvar = model(data)
        
        loss = model.loss(recon_batch, data, mu, logvar)
        loss.backward() # Calc grads
        optimizer.step() # Weight update
        if (i % log_interval == 0) and (epoch % 5 ==0):
            #Print progress
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, i * batch_size, batch_size*batches_per_epoch,
                loss.data[0] / len(data)))   

In [110]:
for epoch in range(1, 70):
    train(epoch)

Train Epoch: 5 [0/25050]	Loss: 0.000313
Train Epoch: 5 [25000/25050]	Loss: 0.000309
Train Epoch: 10 [0/25050]	Loss: 0.000310
Train Epoch: 10 [25000/25050]	Loss: 0.000313
Train Epoch: 15 [0/25050]	Loss: 0.000311
Train Epoch: 15 [25000/25050]	Loss: 0.000310
Train Epoch: 20 [0/25050]	Loss: 0.000309
Train Epoch: 20 [25000/25050]	Loss: 0.000309
Train Epoch: 25 [0/25050]	Loss: 0.000310
Train Epoch: 25 [25000/25050]	Loss: 0.000310
Train Epoch: 30 [0/25050]	Loss: 0.000312
Train Epoch: 30 [25000/25050]	Loss: 0.000310
Train Epoch: 35 [0/25050]	Loss: 0.000310
Train Epoch: 35 [25000/25050]	Loss: 0.000310
Train Epoch: 40 [0/25050]	Loss: 0.000310
Train Epoch: 40 [25000/25050]	Loss: 0.000313
Train Epoch: 45 [0/25050]	Loss: 0.000310
Train Epoch: 45 [25000/25050]	Loss: 0.000311
Train Epoch: 50 [0/25050]	Loss: 0.000309
Train Epoch: 50 [25000/25050]	Loss: 0.000312
Train Epoch: 55 [0/25050]	Loss: 0.000310
Train Epoch: 55 [25000/25050]	Loss: 0.000308
Train Epoch: 60 [0/25050]	Loss: 0.000311
Train Epoch: 60

In [112]:
def construct_numvec(z = None):
    out = np.zeros((1, representation_size))
    out[:representation_size] = 1.
    if z is None:
        return(out)
    else:
        for i in range(len(z)):
            out[:,i] = z[i]
        return(out)

In [113]:
# Splitting for random selection 
_, X_rem = train_test_split(X_train_trun,test_size=0.20, random_state=19)

In [114]:
d = torch.Tensor(X_rem.values)
data = Variable(d, requires_grad=False)
model.eval()
z_dataset = model.rget_z(data)

In [115]:
import numpy as np
z=[]
# Get the z repr
z=z_dataset[:,0].data.numpy()
z1 = z_dataset[:,1].data.numpy

X_rem['z1'] = z
X_rem['z2'] = z1

C:\Users\Swapnilbp\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Swapnilbp\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [116]:
# Calc recon err
def recon_error_cal(actual,predictions):
    recon_error = np.mean(np.power(actual - predictions, 2))
    recon_array = np.power(actual - predictions, 2)
    max_col = np.argmax(recon_array)
    return recon_error, max_col

In [117]:
recon_on_train = [] 
max_contributor =[]
for i in range(X_rem.shape[0]): # For each row in tr
    #z1 = X_rem.iloc[i]['z1']
    z1 = z_dataset[:,0][i]
    #z2 = X_rem.iloc[i]['z2']
    z2 = z_dataset[:,0][i]
    z_ = torch.cat([z1, z2])   # Define z vector
    
    test_pred = model.decode(z_)
    test_pred = test_pred.data.numpy()
    transpose = test_pred.T
    transpose = np.squeeze(transpose) # Decoder predictions
    actuals = X_rem.iloc[i][features]

    recon_error, max_col = recon_error_cal(actuals, transpose)
    recon_on_train.append(recon_error)
    max_contributor.append(max_col[0])

C:\Users\Swapnilbp\Anaconda3\envs\tf-gpu\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [118]:
test_pred

array([0.02269214, 0.02540397, 0.02457797, 0.02462611, 0.0246597 ,
       0.08123855, 0.02361177, 0.49336454, 0.48685247, 0.02334212,
       0.02396036, 0.0243868 ], dtype=float32)

In [119]:
import numpy as np
recon_train_se = pd.Series(recon_on_train)
print("95th percentile recon error", np.percentile(recon_train_se, 95))
anomaly_threshold =  np.percentile(recon_train_se, 95)
print('Selected anomaly threshold: ', anomaly_threshold)

90th percentile recon error 0.03356949664021628
95th percentile recon error 0.038235091257865526
Selected anomaly threshold:  0.038235091257865526


In [160]:
z_test = model.rget_z(Variable(torch.Tensor(X_test.values))) 
z_test = z_test.data.numpy()

In [167]:
recon_on_test = []
max_contributor_test =[]

for i in range(z_test.shape[0]): # For each row
    z_ = Variable(torch.Tensor(z_test[i]))  # Define z vector
    test_pred = model.decode(z_)
    test_pred=test_pred.data.numpy()
    transpose = test_pred.T
    transpose = np.squeeze(transpose) #Decode predictions
    actuals = X_test.iloc[i]

    recon_error , max_col = recon_error_cal(actuals, transpose) # Error calculations 
    # Add this error to recon list
    recon_on_test.append(recon_error)
    max_contributor_test.append(max_col)
    

recon_test_se = pd.Series(recon_on_test)
recon_test_se.shape

max_col_test = pd.Series(max_contributor_test)
max_col_test.shape

C:\Users\Swapnilbp\Anaconda3\envs\tf-gpu\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


(324,)

In [168]:
anomaly_count = 0
for i in range(len(recon_test_se)):
    if recon_test_se[i] > anomaly_threshold:
        anomaly_count +=1
print(anomaly_count/X_test.shape[0])

0.05864197530864197


In [179]:
X_test['recon_error'] = recon_on_test

In [180]:
errs = X_test[X_test['recon_error'] > anomaly_threshold]
errs.sort_values('recon_error', ascending=False)

,High,Low,Last,Bid,Ask,Volume,VWAP,Day,Day_of_week,3rd Closing Avg,5th Closing Avg,10th Closing Avg,recon_error
1324,1.000000,1.000000,0.984502,0.986347,0.984551,0.073541,1.000000,0.566667,0.000000,0.973398,0.956703,0.954588,0.071988
1330,0.799016,0.728683,0.759133,0.760018,0.759149,0.171495,0.771382,0.766667,1.000000,0.772154,0.847341,0.969844,0.057393
168,0.006897,0.007836,0.008559,0.008541,0.008466,0.609295,0.006538,0.166667,0.000000,0.006731,0.007691,0.009030,0.052084
1366,0.605367,0.612044,0.604973,0.606066,0.604967,0.075228,0.608543,0.933333,0.000000,0.595620,0.611736,0.651572,0.049912
1365,0.586929,0.582301,0.592415,0.593307,0.592405,0.078105,0.586524,0.900000,1.000000,0.585961,0.602590,0.649281,0.048451
1401,0.580788,0.595356,0.594122,0.595055,0.594382,0.049800,0.586099,0.133333,0.000000,0.590761,0.594335,0.600371,0.043106
1545,0.415724,0.434528,0.420875,0.421882,0.421141,0.024857,0.423584,0.933333,1.000000,0.425306,0.436698,0.446607,0.043097
1247,0.213393,0.221902,0.219893,0.219870,0.219797,0.050632,0.215954,0.000000,1.000000,0.217049,0.221802,0.220322,0.042878
975,0.038736,0.043233,0.041834,0.041927,0.041749,0.028107,0.040377,0.000000,1.000000,0.040898,0.041752,0.041523,0.042570
1372,0.476959,0.437858,0.475477,0.475455,0.475440,0.126082,0.461796,0.100000,1.000000,0.468982,0.506775,0.589958,0.041825
